In [7]:
import os
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import (
    TextLoader, PyPDFLoader, CSVLoader, Docx2txtLoader,
    UnstructuredPowerPointLoader, UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader, UnstructuredExcelLoader
)
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle

In [8]:
# OpenAI API 키 설정 (실제 사용시에는 환경 변수 등을 통해 안전하게 관리해야 합니다)
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [9]:
# 전역 설정 변수 정의
CHUNK_SIZE = 500  # 텍스트를 나눌 때 각 조각의 크기
CHUNK_OVERLAP = 50  # 텍스트 조각 간 겹치는 부분의 크기
TEMPERATURE = 0.1  # 모델의 창의성 정도 (0에 가까울수록 일관된 답변, 1에 가까울수록 다양한 답변)
MAX_TOKENS = 512  # 생성할 텍스트의 최대 길이
TOP_P = 0.95  # 샘플링에 사용할 누적 확률의 임계값
REPETITION_PENALTY = 1.15  # 반복을 줄이기 위한 페널티 (값이 클수록 반복이 줄어듦)
BATCH_SIZE = 1000  # 데이터 처리 시 한 번에 처리할 항목의 수
NUM_RETRIEVED_DOCUMENTS = 3  # 질문에 답변할 때 참조할 문서의 수
EMBEDDING_MODEL = "sentence-transformers/distiluse-base-multilingual-cased-v2"  # 텍스트를 벡터로 변환하는 데 사용할 모델
MEMORY_FILE = "chat_history.json"  # 대화 기록을 저장할 파일 이름
MODEL_NAME = "gpt-4o-mini"  # 사용할 OpenAI 모델 이름
SEARCH_TYPE = "similarity"  # 'similarity' 또는 'mmr'
SEARCH_KWARGS = {"k": NUM_RETRIEVED_DOCUMENTS}  # 검색 관련 추가 매개변수
PROMPT_TEMPLATE_PATH = "prompts/Genaral_prompt.txt" # 프롬프트 파일 경로를 전역 변수로 설정

In [10]:
class DataLoader:
    """
    다양한 형식의 문서를 로드하는 클래스입니다.
    지원하는 파일 형식: txt, pdf, csv, xlsx, xls, docx, pptx, html, md
    """
    def __init__(self):
        # 파일 확장자와 해당 로더 클래스를 매핑합니다.
        self.loaders = {
            '.txt': TextLoader,
            '.pdf': PyPDFLoader,
            '.csv': CSVLoader,
            '.xlsx': UnstructuredExcelLoader,
            '.xls': UnstructuredExcelLoader,
            '.docx': Docx2txtLoader,
            '.pptx': UnstructuredPowerPointLoader,
            '.html': UnstructuredHTMLLoader,
            '.md': UnstructuredMarkdownLoader
        }

    def load(self, path):
        """
        주어진 경로의 파일 또는 디렉토리에서 문서를 로드합니다.
        
        :param path: 로드할 파일 또는 디렉토리 경로
        :return: 로드된 문서 리스트
        :raises ValueError: 경로가 존재하지 않거나 지원하지 않는 경우
        """
        # 경로를 정규화하고 사용자 홈 디렉토리(~)를 확장합니다.
        path = os.path.normpath(os.path.expanduser(path))
        
        # 경로가 존재하는지 확인합니다.
        if not os.path.exists(path):
            raise ValueError(f"경로가 존재하지 않습니다: {path}")
        
        # 파일인 경우 단일 파일을 로드합니다.
        if os.path.isfile(path):
            return self._load_file(path)
        # 디렉토리인 경우 디렉토리 내의 모든 파일을 로드합니다.
        elif os.path.isdir(path):
            return self._load_directory(path)
        # 파일도 디렉토리도 아닌 경우 에러를 발생시킵니다.
        else:
            raise ValueError(f"지원하지 않는 경로입니다: {path}")


    def _load_file(self, file_path):
        """
        단일 파일을 로드합니다.
        
        :param file_path: 로드할 파일의 경로
        :return: 로드된 문서
        :raises ValueError: 지원하지 않는 파일 형식인 경우
        """
        # 파일 확장자를 소문자로 변환하여 가져옵니다.
        _, ext = os.path.splitext(file_path.lower())
        
        # 지원하지 않는 파일 형식인 경우 에러를 발생시킵니다.
        if ext not in self.loaders:
            raise ValueError(f"지원하지 않는 파일 형식입니다: {ext}")
        
        # 적절한 로더를 사용하여 파일을 로드합니다.
        loader = self.loaders[ext](file_path)
        return loader.load()

    def _load_directory(self, dir_path):
        documents = []
        for root, _, files in os.walk(dir_path):
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    documents.extend(self._load_file(file_path))
                except ValueError as e:
                    print(f"경고: {file_path} 로딩 중 오류 발생 - {str(e)}")
        return documents

    def _load_directory(self, dir_path):
        """
        디렉토리 내의 모든 지원되는 파일을 로드합니다.
        
        :param dir_path: 로드할 디렉토리 경로
        :return: 로드된 문서 리스트
        """
        documents = []
        # 디렉토리를 재귀적으로 순회하며 모든 파일을 처리합니다.
        for root, _, files in os.walk(dir_path):
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    # 각 파일을 로드하여 문서 리스트에 추가합니다.
                    documents.extend(self._load_file(file_path))
                except ValueError as e:
                    # 지원하지 않는 파일 형식 등의 오류 발생 시 경고 메시지를 출력합니다.
                    print(f"경고: {file_path} 로딩 중 오류 발생 - {str(e)}")
        return documents

def load_prompt_template(file_path: str) -> str:
    """
    프롬프트 템플릿 파일을 로드합니다.
    
    :param file_path: 프롬프트 템플릿 파일 경로
    :return: 로드된 프롬프트 템플릿 문자열
    """
    try:
        # 파일을 열어 내용을 읽습니다.
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            # 필요한 변수({context}와 {question})가 템플릿에 포함되어 있는지 확인합니다.
            if "{context}" not in content or "{question}" not in content:
                raise ValueError("프롬프트 템플릿에 {context}와 {question} 변수가 모두 포함되어야 합니다.")
            return content
    except FileNotFoundError:
        # 파일을 찾을 수 없는 경우 경고 메시지를 출력하고 기본 템플릿을 반환합니다.
        print(f"프롬프트 템플릿 파일을 찾을 수 없습니다: {file_path}")
        return """다음 맥락을 사용하여 질문에 답하세요:
        {context}

        인간: {question}
        AI:"""

In [11]:
class ChatBot:
    """
    대화형 AI 챗봇 클래스입니다.
    문서를 로드하고 벡터화하여 질문에 답변합니다.
    """

    def __init__(self, data_path):
        """
        ChatBot 객체를 초기화합니다.

        :param data_path: 로드할 문서 데이터의 경로
        """
        self.data_path = data_path
        self.embeddings = OpenAIEmbeddings()  # 텍스트를 벡터로 변환하는 임베딩 모델
        self.vectorstore = None  # 문서 벡터를 저장할 벡터 저장소
        self.qa_chain = None  # 질문-답변 체인
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)  # 대화 기록을 저장할 메모리

    def setup(self):
        """
        챗봇 시스템을 설정합니다.
        문서를 로드하고, 벡터화하여 저장소를 생성하고, QA 체인을 설정합니다.
        """
        try:
            print("데이터 로딩 중...")
            loader = DataLoader()
            documents = loader.load(self.data_path)  # 문서 로드

            print("텍스트 분할 중...")
            # 긴 문서를 작은 청크로 분할
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
            chunks = text_splitter.split_documents(documents)

            print("벡터 저장소 생성 중...")
            # 분할된 텍스트 청크를 벡터화하여 FAISS 인덱스에 저장
            self.vectorstore = FAISS.from_documents(chunks, self.embeddings)

            print("QA 체인 설정 중...")
            # OpenAI 언어 모델 초기화
            llm = ChatOpenAI(model_name=MODEL_NAME, temperature=TEMPERATURE, max_tokens=MAX_TOKENS)
            
            # 프롬프트 템플릿 로드 및 설정
            prompt_template = load_prompt_template(PROMPT_TEMPLATE_PATH)
            PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
            
            # ConversationalRetrievalChain 설정
            self.qa_chain = ConversationalRetrievalChain.from_llm(
                llm=llm,
                retriever=self.vectorstore.as_retriever(search_kwargs={"k": NUM_RETRIEVED_DOCUMENTS}),
                memory=self.memory,
                combine_docs_chain_kwargs={"prompt": PROMPT},
                chain_type="stuff"  # 'stuff' 방식으로 문서를 처리 (모든 관련 문서를 하나의 컨텍스트로 결합)
            )
            print("설정 완료!")
        except Exception as e:
            print(f"설정 중 오류 발생: {str(e)}")
            raise

    def ask(self, query: str):
        """
        주어진 질문에 대한 답변을 생성합니다.

        :param query: 사용자의 질문
        :return: 챗봇의 답변
        """
        if not self.qa_chain:
            self.setup()  # QA 체인이 설정되지 않은 경우 설정을 실행
        try:
            response = self.qa_chain({"question": query})  # QA 체인을 사용하여 질문에 답변
            return response['answer']
        except Exception as e:
            print(f"질문 처리 중 오류 발생: {str(e)}")
            return "죄송합니다. 질문을 처리하는 중에 오류가 발생했습니다."

    def chat(self):
        """
        대화형 인터페이스를 시작합니다.
        사용자의 입력을 받아 답변을 생성하고 출력합니다.
        """
        print("챗봇이 준비되었습니다. '종료'를 입력하면 대화를 마칩니다.")
        while True:
            query = input("질문을 입력하세요: ")
            if query.lower() == '종료':
                break
            answer = self.ask(query)
            print("\n답변:", answer, "\n")
        print("대화를 종료합니다.")

In [12]:
def main():
    data_path = input("데이터 파일 또는 폴더 경로를 입력하세요: ")
    try:
        chatbot = ChatBot(data_path)
        chatbot.chat()
    except ValueError as e:
        print(f"오류 발생: {str(e)}")

if __name__ == "__main__":
    main()

챗봇이 준비되었습니다. '종료'를 입력하면 대화를 마칩니다.


Ignoring wrong pointing object 69 0 (offset 0)
Ignoring wrong pointing object 312 0 (offset 0)
Ignoring wrong pointing object 313 0 (offset 0)
Ignoring wrong pointing object 314 0 (offset 0)
Ignoring wrong pointing object 317 0 (offset 0)
Ignoring wrong pointing object 332 0 (offset 0)
Ignoring wrong pointing object 345 0 (offset 0)
Ignoring wrong pointing object 346 0 (offset 0)
Ignoring wrong pointing object 349 0 (offset 0)
Ignoring wrong pointing object 350 0 (offset 0)


데이터 로딩 중...
텍스트 분할 중...
벡터 저장소 생성 중...
QA 체인 설정 중...
설정 완료!


/Users/joyong-eun/Desktop/Dev/Langchain/bible/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(



답변: 맥 환경에서 라마(LLaMA)를 랭체인(Chain)으로 활용하기 위해서는 다음 단계를 따라야 합니다:

1. **필수 소프트웨어 설치**:
   - Python이 설치되어 있어야 합니다. Python 3.7 이상을 권장합니다.
   - 필요한 패키지를 설치하기 위해 `pip`를 사용합니다. 다음 명령어를 터미널에 입력하여 `langchain`과 `torch`를 설치하세요:
     ```bash
     pip install langchain torch
     ```

2. **LLaMA 모델 다운로드**:
   - LLaMA 모델을 다운로드하려면 Meta의 공식 웹사이트에서 모델을 요청하고, 다운로드 링크를 받아야 합니다. 모델 파일을 로컬 머신에 저장합니다.

3. **모델 로드 및 설정**:
   - Python 스크립트에서 LLaMA 모델을 로드합니다. 예를 들어, 다음과 같은 코드를 사용할 수 있습니다:
     ```python
     from langchain.llms import Llama

     # 모델 경로를 지정합니다.
     model_path = "path/to/llama/model"
     llama_model = Llama(model_path=model_path)
     ```

4. **랭체인 설정**:
   - 랭체인을 설정하여 LLaMA 모델을 사용할 수 있도록 합니다. 예를 들어, 다음과 같이 간단한 체인을 만들 수 있습니다:
     ```python
     from langchain.chains import SimpleChain

     chain = SimpleChain(llm=llama_model)
     ```

5. **질문 및 응답**:
   - 이제 체인을 사용하여 질문을 하고 응답을 받을 수 있습니다. 예를 들어:
     ```python
     response = chain.run("당신의 질문을 여기에 입력하세요.")
     print(response)
     ```

6.